In [26]:
import pandas as pd
import numpy as np
from main import *

print(features_df.iloc[40])
unseen_data = features_df.iloc[[40,70,3]]
unseen_data_x = unseen_data.drop(columns='chemistry')
unseen_data_y = unseen_data['chemistry']
features_df.drop([40,70,3])

Vmax                            3.600653
Vmin                            1.994813
Vrange                           1.60584
rest_voltage_after_discharge    2.700271
dvdt_discharge                 -0.003613
chemistry                            LFP
Name: 40, dtype: object


,Vmax,Vmin,Vrange,rest_voltage_after_discharge,dvdt_discharge,chemistry
0,4.200795,1.999667,2.201128,2.847339,-0.001893,LCO
1,4.200795,1.999362,2.201433,2.839618,-0.001929,LCO
2,4.200795,1.999057,2.201739,2.833239,-0.001965,LCO
4,4.200795,2.864309,1.336486,NaN,NaN,LCO
5,4.202674,1.998794,2.203880,2.861118,-0.001960,LCO
...,...,...,...,...,...,...
85,4.201031,1.998680,2.202351,2.738758,-0.002644,NMC
86,4.201031,1.998070,2.202961,2.740283,-0.002773,NMC
87,4.201031,1.997459,2.203571,2.741870,-0.002632,NMC
88,4.201031,1.998680,2.202351,2.743793,-0.002866,NMC


In [32]:

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = features_df.drop(columns=["chemistry"]) 
y = features_df["chemistry"]

imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X),columns=X.columns, index= X.index)


X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=4)
rf = RandomForestClassifier(n_estimators=100, random_state=4)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(f"Accuracy:\n {accuracy_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")
print(f"Classification Report: \n{classification_report(y_test, y_pred)}")
importance_df = pd.DataFrame({
    "feature": X.columns,
    "importance": rf.feature_importances_
}).sort_values(by="importance", ascending=False)

print(importance_df)

unseen_pred = rf.predict(unseen_data_x)
unseen_importance = pd.DataFrame({"feature": unseen_data_x.columns,
                                  "importance": rf.feature_importances_}).sort_values(by="importance", ascending=False)
print(unseen_importance)
print("Accuracy: \n", accuracy_score(unseen_data_y, unseen_pred))
print("Confusion Matrix: \n", confusion_matrix(unseen_data_y, unseen_pred))
print("Classification Report: \n", classification_report(unseen_data_y, unseen_pred))


Accuracy:
 1.0
Confusion Matrix:
 [[6 0 0]
 [0 5 0]
 [0 0 7]]
Classification Report: 
              precision    recall  f1-score   support

         LCO       1.00      1.00      1.00         6
         LFP       1.00      1.00      1.00         5
         NMC       1.00      1.00      1.00         7

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

                        feature  importance
0                          Vmax    0.294250
3  rest_voltage_after_discharge    0.281887
4                dvdt_discharge    0.179286
2                        Vrange    0.154854
1                          Vmin    0.089723
                        feature  importance
0                          Vmax    0.294250
3  rest_voltage_after_discharge    0.281887
4                dvdt_discharge    0.179286
2                        Vrange    0.154854
1                          Vmin    0.089723
Accur